# 1. Your first database
**In this chapter, you'll create your very first database with a set of simple SQL commands. Next, you'll migrate data from existing flat tables into that database. You'll also learn how meta-information about a database can be queried.**

- Instructor: Timo Grossenbacher, data journalist in Switzerland

## Introduction to relational databases
In this course, you will see why using relational databases has many advantages over using flat files like CSVs or Excel sheets. You'll learn how to create such databases, and bring into force their most prominent features.

###  Investigating universities in Switzerland
Let me tell you a little story first. As a data journalist, I try to uncover corruption, misconduct and other newsworthy stuff with data. A couple of years ago I researched secondary employment of Swiss university professors. It turns out a lot of them have more than one side job besides their university duty, being paid by big companies like banks and insurances. So I discovered more than 1500 external employments and visualized them in an interactive graphic.
Also, I had to account for certain specialties, for example, that a professor can work for different universities; or that a third-party company can have multiple professors working for them. In order to analyze the data, I needed to make sure its quality was good and stayed good throughout the process. That's why I stored my data in a database, whose quite complex design you can see in the right graphic. All these rectangles were turned into database tables.

###  A relational database:
But why did I use a database? A database models **real-life *entities*** like `professors` and `universities` by storing them in **tables**. Each table only contains data from a single entity type. This **reduces redundancy** by storing entities only once – for example, there only needs to be one row of data containing the details of a certain company. Lastly, a database can be used to model *relationships* between entities. You can define exactly how entities relate to each other. For instance, a `professor` can work at multiple `universities` and `companies`, while a `company` can employ more than one `professor`.

### Throughout this course you will:
Throughout this course, you will actually **work with the same real-life data used during my investigation**. You'll start from a single table of data and build a full-blown relational database from it, column by column, table by table. 

By doing so, you'll get to know ***constraints***, ***keys***, and ***referential integrity***. These three concepts help preserve data quality in databases. By the end of the course, you'll know how to use them. In order to get going, you'll just need a basic understanding of SQL – which can also be used to build and maintain databases, not just for querying data.

### Your first duty: Have a look at the PostgreSQL database
I've already created a single PostgreSQL database table containing all the raw data for this course. In the next few exercises, I want you to have a look at that table. For that, you'll need to retrieve your SQL knowledge and query the `information_schema` database, which is available in PostgreSQL by default.

```sql
SELECT table_schema, table_name
FROM information_schema.tables;
```

`information_schema` is actually some sort of meta-database that holds information about your current database. It's not PostgreSQL specific and also available in other database management systems like MySQL or SQL Server. This `information_schema` database holds various information in different tables, for example in the `tables` table.

```
   table_schema   |         table_name
------------------|-----------------------------
pg_catalog        | pg_statistic
pg_catalog        | pg_type
pg_catalog        | pg_policy
pg_catalog        | pg_authid
pg_catalog        | pg_shadow
public            | university_professors
pg_catalog        | pg_settings
...
```

### Have a look at the columns of a certain table
`information_schema` also holds information about columns in the `columns` table. Once you know the name of a table, you can query its columns by accessing the `columns` table. 
```sql
SELECT table_name, column_name, data_type
FROM information_schema.columns
WHERE table_name = 'pg_congif';
```
```
table_name | columns_name | data_type
-----------|--------------|-----------
pg_config  | name         | text
pg_config  | setting      | text
```

Here, for example, you see that the system table `pg_config` has only two columns – supposedly for storing name-value pairs.

## Query information_schema with SELECT
`information_schema` is a meta-database that holds information about your current database. `information_schema` has multiple tables you can query with the known `SELECT * FROM syntax`:

    - tables: information about all tables in your current database
    - columns: information about all columns in all of the tables in your current database

In this exercise, you'll only need information from the `'public'` schema, which is specified as the column `table_schema` of the `tables` and `columns` tables. The `'public'` schema holds information about user-defined tables and databases. The other types of `table_schema` hold system information – for this course, you're only interested in user-defined stuff.

- Get information on all table names in the current database, while limiting your query to the `'public'` `table_schema`.

```SQL
-- Query the right table in information_schema
SELECT table_name 
FROM information_schema.tables
-- Specify the correct table_schema value
WHERE table_schema = 'public';
```

```
table_name
----------------------
university_professors
```

- Now have a look at the columns in `university_professors` by selecting all entries in `information_schema.columns` that correspond to that table.

```sql
-- Query the right table in information_schema to get columns
SELECT column_name, data_type 
FROM information_schema.columns
WHERE table_name = 'university_professors' 
    AND table_schema = 'public';
```

```
column_name          | data_type
---------------------|----------
firstname            | text
lastname             | text
university           | text
university_shortname | text
university_city      | text
function             | text
organization         | text
organization_sector  | text
```

- Finally, print the first five rows of the `university_professors` table.

```sql
-- Query the first five rows of our table
SELECT * 
FROM university_professors 
LIMIT 5;
```

```
firstname       | lastname | university   | university_shortname | university_city | function                                         | organization                   | organization_sector
----------------|----------|--------------|----------------------|-----------------|--------------------------------------------------|--------------------------------|------------------------------
Karl            | Aberer   | ETH Lausanne | EPF                  | Lausanne        | Chairman of L3S Advisory Board                   | L3S Advisory Board             | Education & research
Karl            | Aberer   | ETH Lausanne | EPF                  | Lausanne        | Member Conseil of Zeno-Karl Schindler Foundation | Zeno-Karl Schindler Foundation | Education & research
Karl            | Aberer   | ETH Lausanne | EPF                  | Lausanne        | Member of Conseil Fondation IDIAP                | Fondation IDIAP                | Education & research
Karl            | Aberer   | ETH Lausanne | EPF                  | Lausanne        | Panel Member                                     | SNF Ambizione Program          | Education & research
Reza Shokrollah | Abhari   | ETH Zürich   | ETH                  | Zurich          | Aufsichtsratsmandat                              | PNE Wind AG                    | Energy, environment & mobility
```

*You're now familiar with the pre-existing `university_professors` table, which holds information on all kinds of entities. You'll migrate data from this table to other tables in the upcoming lessons.*

---
## Tables: At the core of every database
Let's delve into one of the most important concepts behind databases: **tables**.

### Redundancy in the university_professors table
You might have noticed that there's some redundancy in the `university_professors` table. Let's have a look at the first three records, for example.
```sql
SELECT * FROM
FROM university_professors
LIMIT 3;
```
```
-[ RECORD 1 ]--------+-------------------------------------------------
firstname            | Karl
lastname             | Aberer
university           | ETH Lausanne
university_shortname | EPF
university_city      | Lausanne
function             | Chairman of L3S Advisory Board
organization         | L3S Advisory Board
organization_sector  | Education & research
-[ RECORD 2 ]--------+-------------------------------------------------
firstname            | Karl
lastname             | Aberer
university           | ETH Lausanne
university_shortname | EPF
university_city      | Lausanne
function             | Member Conseil of Zeno-Karl Schindler Foundation
organization         | Zeno-Karl Schindler Foundation
organization_sector  | Education & research
-[ RECORD 3 ]--------+-------------------------------------------------
firstname            | Karl
lastname             | Aberer
university           | ETH Lausanne
university_shortname | EPF
university_city      | Lausanne
function             | Member of Conseil Fondation IDIAP
organization         | Fondation IDIAP
organization_sector  | Education & research
```

As you can see, this professor is repeated in the first three records. Also, his university, the "`ETH Lausanne`, is repeated a couple of times – because he only works for this university. However, he seems to have affiliations with at least three different organizations. So, there's a certain redundancy in that table. The reason for this is that the table actually contains entities of at least three different types. Let's have a look at these entity types.

Actually the table stores professors, highlighted in blue, universities, highlighted in green, and organizations, highlighted in brown. There's also this column called "function" which denotes the role the professor plays at a certain organization. More on that later.

~### Currently: One "entity type" in the database~
~Let's look at the current database once again. The graphic used here is called an entity-relationship diagram. Squares denote so-called entity types, while circles connected to these denote attributes (or columns). So far, we have only modeled one so-called entity type – `university_professors`. However, we discovered that this table actually holds many different entity types...~

~### A better database model with three entity types~
~...so this updated entity-relationship model on the right side would be better suited. It represents three entity types, "professors", "universities", and "organizations" in their own tables, with respective attributes. This reduces redundancy, as professors, unlike now, need to be stored only once. Note that, for each professor, the respective university is also denoted through the "university_shortname" attribute. However, one original attribute, the "function", is still missing.~

~### A better database model with four entity types~
~As you know, this database contains affiliations of professors with third-party organizations. The attribute "function" gives some extra information to that affiliation. For instance, somebody might act as chairman for a certain third-party organization. So the best idea at the moment is to store these affiliations in their own table – it connects professors with their respective organizations, where they have a certain function.~

### Create new tables with CREATE TABLE
The first thing you need to do now is to create four empty tables for professors, universities, organizations, and affiliations. This is quite easy with SQL – you'll use the `CREATE TABLE` command for that. At the minimum, this command requires a table name and one or more columns with their respective data types.
```sql
CREATE TABLE table_name (
    column_a data_type,
    column_b data_type,
    column_c data_type,
    );
```

For example, you could create a `weather` table with three aptly named columns. After each column name, you must specify the data type. 
```sql
CREATE TABLE weather (
    clouds text,
    temperature numeric,
    weather_station char(5)
    );
```
There are many different types, and you will discover some in the remainder of this course. For example, you could specify a text column, a numeric column, and a column that requires fixed-length character strings with 5 characters each.

## CREATE TABLEs
You'll now start implementing a better database model. For this, you'll create tables for the `professors` and `universities` entity types. The other tables will be created for you.

The syntax for creating simple tables is as follows:
```sql
CREATE TABLE table_name (
    column_a data_type,
    column_b data_type, 
    column_c data_type
);
```
- Create a table `professors` with two `text` columns: `firstname` and `lastname`.

```sql
-- Create a table for the professors entity type
CREATE TABLE professors (
 firstname text,
 lastname text
);
```

- Create a table `universities` with three `text` columns: `university_shortname`, `university`, and `university_city`.

```sql
-- Create a table for the universities entity type
CREATE TABLE universities (
    university_shortname text,
    university text,
    university_city text
);
```

## ADD a COLUMN with ALTER TABLE
- Alter `professors` to add the text column `university_shortname`.

```sql
-- Add the university_shortname column
ALTER TABLE professors
ADD COLUMN university_shortname text;
```

---
## Update your database as the structure changes
###  Only store DISTINCT data in the new tables
One advantage of splitting up `university_professors` into several tables is the reduced redundancy. As of now, `university_professors` holds 1377 entries.
```sql
SELECT COUNT(*)
FROM university_professors;
```
```
count
-----
1377
```

However, there are only 1287 distinct organizations, as this query shows. Therefore, you only need to store 1287 distinct organizations in the new `organizations` table.
```sql
SELECT COUNT(DISTINCT organization)
FROM university_professors;
```
```
count
-----
1287
```

### INSERT DISTINCT records INTO the new tables
In order to copy data from an existing table to a new one, you can use the `INSERT INTO SELECT DISTINCT` pattern. After `INSERT INTO`, you specify the name of the target table – `organizations` in this case. Then you select the columns that should be copied over from the source table – `unviversity_professors` in this case. You use the `DISTINCT` keyword to only copy over distinct organizations. 
```sql
INSERT INTO organizations
SELECT DISTINCT organization,
    organization_sector
FROM university_professors;
```
```
Output: INSERT 0 1287
```

As the output shows, only 1287 records are inserted into the `organizations` table. 

If you just used `INSERT INTO SELECT`, **without** the `DISTINCT` keyword, duplicate records would be copied over as well. 
```sql
INSERT INTO organizations
SELECT organization,
    organization_sector
FROM university_professors;
```
```
Output: INSERT 0 1377
```

In the following exercises, you will migrate your data to the four new tables.

### The INSERT INTO statement
By the way, this is the normal use case for `INSERT INTO` – where you insert values manually. `INSERT INTO` is followed by the table name and an optional list of columns which should be filled with data. Then follows the `VALUES` keyword and the actual values you want to insert.
```sql
INSERT INTO table_name (column_a, column_b)
VALUES ('value_a', 'value_b');
```

### RENAME a COLUMN in affiliations
Before you start migrating the table, you need to fix some stuff! In the last lesson, I created the `affiliations` table for you. Unfortunately, I made a mistake in this process. Can you spot it? 
```sql
CREATE TABLE affiliations (
    firstname text,
    lastname text,
    university_shortname text,
    function text,
    organisation text
);
```

The way the `organisation` column is spelled is not consistent with the American-style spelling of this table, using an `s` instead of a `z`. 

You can correct this with the known `ALTER TABLE` syntax. 
You do this with the `RENAME COLUMN` command by specifying the old column name first and then the new column name, i.e., `RENAME COLUMN old_name TO new_name`.
```sql
ALTER TABLE table_name
RENAME COLUMN old_name TO new_name;
```

### DROP a COLUMN in affiliations
Also, the `university_shortname` column is not even needed here. 
```sql
CREATE TABLE affiliations (
    firstname text,
    lastname text,
    university_shortname text,
    function text,
    organization text
);
```
So I want you to delete it. The syntax for this is again very simple, you use a `DROP COLUMN` command followed by the name of the column. 
```sql
ALTER TABLE table_name
DROP COLUMN column_name;
```
Dropping columns is straightforward when the tables are still empty, so it's not too late to fix this error. But why is it an error in the first place?

### A professor is uniquely identified by firstname, lastname only
Well, I queried the `university_professors` table and saw that there are 551 unique combinations of first names, last names, and associated universities.
```sql
SELECT DISTINCT firstname, lastname, university_shortname
FROM university_professors
ORDER BY lastnames;
```
 I then queried the table again and only looked for unique combinations of first and last names. 
```sql
SELECT DISTINCT firstname, lastname
FROM university_professors
ORDER BY lastnames;
```
 
Turns out, this is also 551 records. This means that the columns `firstname` and `lastname` uniquely identify a professor.

So the `university_shortname` column is not needed in order to reference a professor in the affiliations table. You can remove it, and this will reduce the redundancy in your database again. In other words: The columns `firstname`, `lastname`, `function`, and `organization` are enough to store the affiliation a professor has with a certain organization.

## RENAME and DROP COLUMNs in affiliations
The still empty affiliations table has some flaws. In this exercise, you'll correct them as mentioned above.

You'll use the following queries:

To rename columns:
```sql
ALTER TABLE table_name
RENAME COLUMN old_name TO new_name;
```
To delete columns:
```sql
ALTER TABLE table_name
DROP COLUMN column_name;
```

- Rename the `organisation` column to `organization` in `affiliations`.

```sql
-- Rename the organisation column
ALTER TABLE affiliations
RENAME COLUMN organisation TO organization;
```

- Delete the `university_shortname` column in `affiliations`.

```sql
-- Delete the university_shortname column
ALTER TABLE affiliations
DROP COLUMN university_shortname;
```

## Migrate data with INSERT INTO SELECT DISTINCT
Now it's finally time to migrate the data into the new tables. You'll use the following pattern:

```sql
INSERT INTO ... 
SELECT DISTINCT ... 
FROM ...;
```
It can be broken up into two parts:

*First part*:
```sql
SELECT DISTINCT column_name1, column_name2, ... 
FROM table_a;
```
This selects all distinct values in table `table_a` – nothing new for you.

*Second part*:
```sql
INSERT INTO table_b ...;
```
Take this part and append it to the first, so it inserts all distinct rows from `table_a` into `table_b`.

**One last thing**: It is important that you run all of the code at the same time once you have filled out the blanks.

- Insert all `DISTINCT` professors from `university_professors` into `professors`.

```sql
-- Insert unique professors into the new table
INSERT INTO professors 
SELECT DISTINCT firstname, lastname, university_shortname 
FROM university_professors;
```

- Insert all `DISTINCT` affiliations into `affiliations` from `university_professors`.

```sql
-- Insert unique affiliations into the new table
INSERT INTO affiliations 
SELECT DISTINCT firstname, lastname, function, organization 
FROM university_professors;
```

*There are 1377 distinct combinations of professors and organisations in the dataset. We'll migrate the other two tables `universities` and `organizations` for you. The last thing to do in this chapter is to delete the no longer needed `university_professors` table.*

## Delete tables with DROP TABLE
Obviously, the `university_professors` table is now no longer needed and can safely be deleted.

For table deletion, you can use the simple command:
```sql
DROP TABLE table_name;
```

- Delete the `university_professors` table.

```sql
-- Delete the university_professors table
DROP TABLE university_professors;
```